<a href="https://colab.research.google.com/github/milkyme/CS246/blob/main/CS246_Colab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 4
## Collaborative Filtering

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 25 kB/s 
     |████████████████████████████████| 198 kB 45.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=365f97b2f1e71bdd0f3034796a3f341db7c1d189131c459d7064ad198fa3033f
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

Now we authenticate a Google Drive client to download the filea we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2022-01-19 10:53:56--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  9.16MB/s    in 1.4s    

2022-01-19 10:53:58 (9.16 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [8]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [9]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [10]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [11]:
# YOUR CODE HERE
print('Training set count: {0}, Test set count: {1}, Movie set count: {2}'\
      .format(training.count(), test.count(), items.count()))

Training set count: 80000, Test set count: 20000, Movie set count: 1682


In [20]:
training.take(3)

[Row(user_id=1, item_id=1, rating=5, timestamp=874965758),
 Row(user_id=1, item_id=2, rating=3, timestamp=876893171),
 Row(user_id=1, item_id=3, rating=4, timestamp=878542960)]

Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [12]:
# YOUR CODE HERE
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

# lines = spark.read.text("data/mllib/als/sample_movielens_ratings.txt").rdd
# parts = lines.map(lambda row: row.value.split("::"))
# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
#                                      rating=float(p[2]), timestamp=int(p[3])))
# ratings = spark.createDataFrame(ratingsRDD)
# (training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

Now compute the RMSE on the test dataset.


In [13]:
# YOUR CODE HERE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.1409426520573127


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [15]:
# YOUR CODE HERE
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)

# Generate top 10 movie recommendations for a specified set of users
users = training.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [16]:
userRecs.take(3)

[Row(user_id=1, recommendations=[Row(item_id=1001, rating=6.480434894561768), Row(item_id=1643, rating=6.444374084472656), Row(item_id=1205, rating=6.1051130294799805), Row(item_id=1615, rating=6.001343250274658), Row(item_id=787, rating=6.000215530395508), Row(item_id=900, rating=5.919325828552246), Row(item_id=1449, rating=5.9021315574646), Row(item_id=1475, rating=5.837296962738037), Row(item_id=921, rating=5.8356032371521), Row(item_id=793, rating=5.8017497062683105)]),
 Row(user_id=3, recommendations=[Row(item_id=1206, rating=6.225497245788574), Row(item_id=1184, rating=6.104776382446289), Row(item_id=1159, rating=5.572500228881836), Row(item_id=695, rating=5.480177879333496), Row(item_id=1491, rating=5.251319408416748), Row(item_id=1499, rating=5.248824119567871), Row(item_id=115, rating=5.203904151916504), Row(item_id=952, rating=5.109012603759766), Row(item_id=1098, rating=4.972006797790527), Row(item_id=1070, rating=4.926609039306641)]),
 Row(user_id=5, recommendations=[Row(it

In [19]:
userSubsetRecs.take(3)

[Row(user_id=471, recommendations=[Row(item_id=1311, rating=13.14230728149414), Row(item_id=551, rating=12.208623886108398), Row(item_id=1540, rating=11.247785568237305), Row(item_id=1185, rating=11.086333274841309), Row(item_id=1167, rating=10.788458824157715), Row(item_id=1419, rating=10.608564376831055), Row(item_id=753, rating=10.233625411987305), Row(item_id=1078, rating=10.038732528686523), Row(item_id=1153, rating=9.889566421508789), Row(item_id=967, rating=9.712368965148926)]),
 Row(user_id=463, recommendations=[Row(item_id=745, rating=6.098721027374268), Row(item_id=767, rating=5.918541431427002), Row(item_id=1639, rating=5.875011444091797), Row(item_id=351, rating=5.6903767585754395), Row(item_id=1368, rating=5.457205295562744), Row(item_id=1150, rating=5.409814834594727), Row(item_id=645, rating=5.403030872344971), Row(item_id=899, rating=5.387301921844482), Row(item_id=1166, rating=5.325599670410156), Row(item_id=703, rating=5.324996471405029)]),
 Row(user_id=148, recommend

Once you have working code for each cell above, **head over to Gradescope, carefully read the questions, and submit your solution for this Colab**!